In [ ]:
# 1.Import necessary packages
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments, pipeline
from datasets import load_dataset
from evaluate import load as load_metric

In [ ]:
# 2.Load cnn_dailymail dataset
dataset = load_dataset("cnn_dailymail", "3.0.0", split="train[:1%]")


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
# 3. Load T5-small model
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

In [ ]:
# 4. Define preprocessing function
def preprocess_function(examples):
    inputs = ["summarize: " + doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["highlights"], max_length=150, truncation=True, padding="max_length")
    
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [ ]:
# 5. Preprocess dataset
train_dataset = dataset.map(preprocess_function, batched=True, remove_columns=["article", "highlights", "id"])

c:\Users\myick\anaconda3\envs\zhou\Lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# 6. Set training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="no",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    num_train_epochs=1,
    weight_decay=0.01,
    save_total_limit=2,
    report_to="none",  
)

In [ ]:
# 7. Train using Trainer API
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)


  0%|          | 0/2154 [00:00<?, ?it/s]

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


{'loss': 2.681, 'grad_norm': 4.6257853507995605, 'learning_rate': 4.976787372330548e-05, 'epoch': 0.01}
{'loss': 2.5356, 'grad_norm': 3.1615967750549316, 'learning_rate': 4.953574744661096e-05, 'epoch': 0.03}
{'loss': 2.4958, 'grad_norm': 3.9571962356567383, 'learning_rate': 4.930362116991643e-05, 'epoch': 0.04}
{'loss': 2.3863, 'grad_norm': 3.3057658672332764, 'learning_rate': 4.9071494893221914e-05, 'epoch': 0.06}
{'loss': 2.2961, 'grad_norm': 2.82592511177063, 'learning_rate': 4.883936861652739e-05, 'epoch': 0.07}
{'loss': 2.2758, 'grad_norm': 3.300835132598877, 'learning_rate': 4.860724233983287e-05, 'epoch': 0.08}
{'loss': 2.3751, 'grad_norm': 2.6961417198181152, 'learning_rate': 4.8375116063138345e-05, 'epoch': 0.1}
{'loss': 2.144, 'grad_norm': 3.0563158988952637, 'learning_rate': 4.8142989786443826e-05, 'epoch': 0.11}
{'loss': 2.3914, 'grad_norm': 3.7386505603790283, 'learning_rate': 4.79108635097493e-05, 'epoch': 0.13}
{'loss': 2.3225, 'grad_norm': 3.1724650859832764, 'learning

  0%|          | 0/359 [00:00<?, ?it/s]

In [ ]:
# 8. Start training   
trainer.train()

In [ ]:
summarization_pipeline = pipeline("summarization", model=model, tokenizer=tokenizer)


test_article = "The US has passed the peak on new coronavirus cases, President Donald Trump said, and he predicted that some states would reopen this month."
summarized_text = summarization_pipeline(test_article, max_length=50, min_length=25, do_sample=False)
print(summarized_text)

In [ ]:
# Evaluate the model
metric = load_metric("rouge")

validation_dataset = load_dataset("cnn_dailymail", "3.0.0", split="validation[:1%]")
validation_dataset = validation_dataset.map(preprocess_function, batched=True, remove_columns=["id"])

for example in validation_dataset:
    inputs = tokenizer("summarize: " + example["article"], return_tensors="pt", max_length=512, truncation=True)
    summary_ids = model.generate(inputs.input_ids, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    predicted_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    metric.add(prediction=predicted_summary, reference=example["highlights"])

# Print ROUGE scores
result = metric.compute()
print(result)
